In [6]:
import os
import argparse

In [15]:
def outer_inner( End_point):
    """
    writes outer and inner in neginex.
  
    Parameters:
    End_point (str): End point that  used  as outer and inner locations
    point: point that  used  as outer and inner locations
  
    Returns:
    string: outer and inner
  
    """
    outer = '## outer users'
    
    inner = "## inner users, has no version because it should be updated instantly"
    
    common = 'proxy_set_header Host $host; \n proxy_set_header X-Real-IP $remote_addr;'\
              '\n proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for' 
    
    point = End_point.split('/')[-1]
    return f'\n {outer:2s} \n location ^~ /ai{End_point} \n  \n proxy_pass $address_parser_upstream:660 {point}; \n {common}\n\n'\
                 f'\n {inner:2s} \n location ^~ {End_point} \n  \n proxy_pass $address_parser_upstream:660 {point}; \n {common}\n'\
                 f' allow 172.16.0.0/12; \n deny all;\n \n'


def server(valid=30, size='20M'):
    return f'  listen 80;\n  server_name 0.0.0.0;\n  resolver 127.0.0.11 valid=30s;\n  client_max_body_size 20M;\n \n'


In [16]:
def title(name):
    """
    writes title in neginex.
  
    Parameters:
    name (str): Titles that are used both as a title and as outer and inner locations

    Returns:
    string: outer and inner
  
    """
    title = f'##{name}'
    return f'{title} \n set ${name}_upstream http://{name};\n \n'


In [17]:
# parser = argparse.ArgumentParser(description='The file  want to make negix from')
# parser.add_argument("docker",help='The file  want to make negix from', default='docker-compose.yml')
# args = parser.parse_args()
# docker = args.docker()

#Creating a dictionary of titles along with endpoints
dic = dict()
with open('docker-compose.yml', encoding="utf-8") as docer_file:
    deta = docer_file.readlines()
    for line in deta:
        count = 0
        #count space 
        for item in line[:5]:
            if item == ' ':
                count += 1
            else:
                break
        #Find based on distance
        if count == 2 :
            line_ = line.strip().strip(':').strip('# ')
            # Add titles as key (along with a list of values that are empty)
            dic[line_] = []
            pass
        
        if 'ENDPOINT'  in line:
                line_end_point = line.strip().strip(':').strip('# ').strip('-')
                end_point = line_end_point.split('=')[-1]
                # Add endpoint as value
                dic[line_].append(end_point)
with open('nginx', 'w+') as nginx:
    nginx.write(server())
    for key, value in dic.items():
        if value:
            nginx.write(title(key))
            for i in range(len(value)):
                nginx.write(outer_inner(value[i]))
      